In [ ]:
import re
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import BertModel, BertTokenizer
import torch
import nltk
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import spacy

In [ ]:
avaliacoes_df = pd.read_csv('/Users/Mariana Nascimento/Modelo_Recomendacao_ECommerce/results/avaliacoes.csv')

In [ ]:
reviews = avaliacoes_df['texto_da_avaliacao']

### 1 - Limpeza de texto

In [ ]:
# Função para remover acentos
def remove_accents(text):
    nfkd_form = unicodedata.normalize('NFKD', text)
    return ''.join([char for char in nfkd_form if not unicodedata.combining(char)])


In [ ]:
# Função de limpeza de texto
def clean_text(text):
    text = remove_accents(text)  # Remover acentos
    text = re.sub(r'<.*?>', '', text)  # Remover tags HTML
    text = re.sub(r'[^a-zA-Z\s]', ' ', text)  # Remover caracteres especiais, exceto espaços
    text = text.lower()  # Converter para minúsculas
    return text

In [ ]:
reviews = reviews.apply(remove_accents)
reviews = reviews.apply(clean_text)

### 1 - Tokenização

Tokenização é o processo de dividir um texto em unidades menores chamadas tokens. Esses tokens podem ser palavras, frases ou até mesmo caracteres, dependendo do nível de granularidade desejado.

In [ ]:
nltk.download('punkt')

In [ ]:
texto = "Processamento de Linguagem Natural com Python é ótimo. NLTK facilita bastante."
palavras = word_tokenize(texto, language='portuguese')
sentencas = sent_tokenize(texto, language='portuguese')

### 2 - Remoção de 'stop words'

In [ ]:
nltk.download('stopwords')

In [ ]:
stop_words = set(stopwords.words('portuguese'))

In [ ]:
def remove_stop_words(text):
    # Tokenizar o texto em palavras
    words = word_tokenize(text, language='portuguese')
    # Filtrar as palavras que não estão na lista de stop words
    filtered_words = [word for word in words if word.lower() not in stop_words]
    # Reconstruir o texto a partir das palavras filtradas
    filtered_text = ' '.join(filtered_words)
    return filtered_text

In [ ]:
reviews = reviews.apply(remove_stop_words)

### 3 - Entidades (Não sera usado nesse modelo)

nlp = spacy.load('pt_core_news_sm')

def extract_entities(text):
    # Processar o texto com o modelo SpaCy
    doc = nlp(text)
    # Extrair entidades nomeadas
    entities = [(entity.text, entity.label_) for entity in doc.ents]
    return entities

texto_exemplo = "O iPhone 12 é um excelente smartphone da Apple. Eu o comprei na loja da Apple em São Paulo."
entidades = extract_entities(texto_exemplo)

### 4 - Stemming

In [ ]:
nltk.download('punkt')
nltk.download('rslp')

In [ ]:
# Inicializar o stemmer para português
stemmer = RSLPStemmer()

def apply_stemming(text):
    # Tokenizar o texto em palavras
    words = word_tokenize(text, language='portuguese')
    # Aplicar o stemmer a cada palavra
    stemmed_words = [stemmer.stem(word) for word in words]
    # Reconstruir o texto a partir das palavras stemmizadas
    stemmed_text = ' '.join(stemmed_words)
    return stemmed_text

In [ ]:
reviews = reviews.apply(apply_stemming)

### 5 - Embeddings contextuais

In [ ]:
# Carregar o tokenizador e o modelo BERT pré-treinado
tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')
model = BertModel.from_pretrained('neuralmind/bert-base-portuguese-cased')

In [ ]:
# Função para vetorizar as reviews usando BERT
def vectorize_reviews(reviews):
    # Tokenizar as reviews
    inputs = tokenizer(reviews, return_tensors='pt', padding=True, truncation=True)

    # Passar as reviews pelo modelo BERT para obter as embeddings
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Obter a saída da camada de embeddings oculta
    embeddings = outputs.last_hidden_state

    return embeddings


In [ ]:
embeddings = vectorize_reviews(reviews)

In [ ]:
embeddings_list = embeddings.tolist()

# Adicionar os embeddings como uma nova coluna no dataframe original
avaliacoes_df['embeddings'] = pd.Series(embeddings_list)

In [ ]:
avaliacoes_df.head(20)